<a href="https://colab.research.google.com/github/GEJ1/buscador-linkedin/blob/main/generar_linkedin_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos sentence-transformers porque no viene en google colab
!pip install sentence-transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
import numpy as np
import json
import pandas as pd
import re

## Cargamos los datos

El archivo `Shares.csv` es uno de los que proporciona Linkedin cuando le pedimos nuestros datos. Para hacer eso podés seguir estas instrucciones: https://www.linkedin.com/help/linkedin/answer/a1340367/descargar-los-datos-de-tu-cuenta?lang=es-ES

In [ ]:

shares_data = pd.read_csv("https://raw.githubusercontent.com/GEJ1/buscador-linkedin/refs/heads/main/data/Shares.csv")
shares_data['ShareCommentary'] = shares_data['ShareCommentary'].fillna('')
# Filtrar por longitud, esto es opcional
shares_data = shares_data[shares_data['ShareCommentary'].str.len() > 20]
shares_data.shape

In [ ]:
shares_data.info()

## Cargamos el modelo y generamos los embeddings

En este caso elegí uno en español. Másinformación: https://huggingface.co/hiiamsid/sentence_similarity_spanish_es

In [ ]:
# Cargamos el modelo
# https://huggingface.co/hiiamsid/sentence_similarity_spanish_es
model_type = 'hiiamsid/sentence_similarity_spanish_es'
model = SentenceTransformer(model_type)

# Definimos el batch size
batch_size = 32
comments = shares_data['ShareCommentary'].tolist()

# Generamos los embeddings en batches
embeddings = []
for i in range(0, len(comments), batch_size):
    batch = comments[i:i + batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

# Convert embeddings to a NumPy array
embeddings = np.array(embeddings)
shares_data['embeddings'] = list(embeddings)


## Código para la búsqueda

Lo importante es la comparación entre la query codificada en el modelo y los embeddings generados previamente.

En este elegí la similaridad coseno para comparar.

In [ ]:
def search_posts(query, data, model, n=5):
    query_embedding = model.encode([query])
    embeddings = np.vstack(data['embeddings'])
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    top_indices = similarities.argsort()[-n:][::-1]  # Top n
    return data.iloc[top_indices]

# Step 6: Perform a Search
query = "quiero aprender python"
results = search_posts(query, shares_data, model,n=10)

# Step 7: Display Results
for i, row in results.iterrows():
    print(f"Post {i + 1}:\n{row['ShareCommentary']}\nLink: {row['ShareLink']}\n{'-'*40}")

## Guardamos la información con los embeddings

Acá usé json porque sino al recuperarlos no funcionaba correctamente. Esto se debe a como guarda Pandas las celdas en formato string. Probablemente sería mejor guardar los embeddings por separado.

In [ ]:
shares_data['embeddings'] = shares_data['embeddings'].apply(lambda x: json.dumps(x.tolist()))
shares_data.to_csv(f'Shares_with_embeddings_{model_type.split("/")[-1]}.csv', index=False)